# Measurement Grouping

Since current quantum hardware is limited to single-qubit projective measurement, only terms commuting within individual qubit's subspace can be measured together. These terms are said to be qubit-wise commuting (QWC). 

Thus, one can not measure the entire electronic Hamiltonian $\hat H$ at once, and instead needs to separate it into fragments. 
$$\hat H = \sum_n \hat H_n$$
where each $\hat H_n$ is a QWC fragment. 

In [ ]:
from utility import * 

Here we use $H_4$ as an example for finding QWC fragments. Notice below that each fragment has the same terms on all qubits.  

To show differences between QWC and more advanced grouping, we didn't use the qubit-tappering techinique shown in step 2.

In [9]:
h4 = get_qubit_hamiltonian(mol='h4', geometry=1, basis='sto3g', qubit_transf='jw')

qwc_list = get_qwc_group(h4)
print('Fragments 1: \n{}\n'.format(qwc_list[4]))
print('Fragments 2:\n{}\n'.format(qwc_list[1]))
print('Number of fragments: {}'.format(len(qwc_list)))

Fragments 1: 
0.2383993319488192 [Z0] +
0.09518660668119278 [Z0 Z2] +
0.09518660668118697 [Z0 Z4] +
0.11120069420693832 [Z0 Z6] +
0.014173745323078192 [Y1 Z2 Y3 Y5 Z6 Y7] +
0.10669392951172978 [Z2 Z4] +
-0.008917138825534097 [Z4] +
0.09542973454772244 [Z4 Z6]

Fragments 2:
0.014173745323078192 [Y0 Z1 Y2 Y4 Z5 Y6]

Number of fragments: 37


By applying extra unitaries, one may rotate more terms of $\hat H$ into a QWC fragment.  

Recall that in digital quantum computing, the expectation value of $\hat H_n$ given a trial wavefunction $|\psi\rangle$ is 
$$ E_n =\ \langle\psi| \hat H_n | \psi\rangle$$
Inserting unitary transformation $\hat U_n$ does not change the expectation value.
$$ E_n =\ \langle\psi| \hat U_n^\dagger \hat U_n \hat H_n \hat U_n^\dagger \hat U_n  |\psi\rangle$$ 
This nonetheless changes the trial wavefunction and the terms to be measured. 
$$ |\psi\rangle \rightarrow \hat U_n |\psi\rangle = |\phi\rangle$$
$$ \hat H_n \rightarrow \hat U_n \hat H_n \hat U_n^\dagger = \hat A_n$$
The transformation of $|\psi \rangle$ can be done on the quantum computer, and the transformation of $\hat H_n$ is possible on the classical computer. 

Now, although $\hat A_n$ needs to be a QWC fragment to be measurable on a quantum computer, $\hat H_n$ does not. 
Instead, if we restrict $\hat U_n$ to be a clifford operation, the terms in $\hat H$ need only mutually commute. 

Here, we obtain measurable parts of $H_4$ by partitioning its terms into mutually commuting fragments. 

In [10]:
comm_groups = get_commuting_group(h4)
print('Number of mutually commuting fragments: {}'.format(len(comm_groups)))
print('The first commuting group')
print(comm_groups[1])

Number of mutually commuting fragments: 6
The first commuting group
-0.24347573739328984 [] +
-0.03422206211800904 [X0 X1 Y4 Y5] +
0.03422206211800904 [X0 Y1 Y4 X5] +
0.014173745323078192 [X0 Z1 Y2 Y4 Z5 X6] +
-0.03361644838364563 [X0 Z1 Z2 X3 Y4 Z5 Z6 Y7] +
0.03361644838364563 [X0 Z1 Z2 Y3 Y4 Z5 Z6 X7] +
0.03422206211800904 [Y0 X1 X4 Y5] +
-0.03422206211800904 [Y0 Y1 X4 X5] +
0.014173745323078192 [Y0 Z1 X2 X4 Z5 Y6] +
0.03361644838364563 [Y0 Z1 Z2 X3 X4 Z5 Z6 Y7] +
-0.03361644838364563 [Y0 Z1 Z2 Y3 X4 Z5 Z6 X7] +
0.09518660668118697 [Z0 Z4] +
-0.03361644838364563 [X1 X2 Y5 Y6] +
0.03361644838364563 [X1 Y2 Y5 X6] +
0.014173745323078192 [X1 Z2 Y3 Y5 Z6 X7] +
0.03361644838364563 [Y1 X2 X5 Y6] +
-0.03361644838364563 [Y1 Y2 X5 X6] +
0.014173745323078192 [Y1 Z2 X3 X5 Z6 Y7] +
0.09518660668118697 [Z1 Z5] +
-0.03691448415649887 [X2 X3 Y6 Y7] +
0.03691448415649887 [X2 Y3 Y6 X7] +
0.03691448415649887 [Y2 X3 X6 Y7] +
-0.03691448415649887 [Y2 Y3 X6 X7] +
0.09542973454771654 [Z2 Z6] +
0.0954297345

To see this fragment is indeed measurable, one can construct the corresponding unitary operator $\hat U_n$.

In [11]:
uqwc = get_qwc_unitary(comm_groups[1])
print('This is unitary, U * U^+ = I ')
print(uqwc * uqwc)

This is unitary, U * U^+ = I 
(0.9999999999999999+0j) []


Applying this unitary gives the qubit-wise commuting form of the first mutually commuting group

In [12]:
qwc = remove_complex(uqwc * comm_groups[1] * uqwc)
print(qwc)

-0.24347573739328962 [] +
0.03361644838364539 [X0 X1 X2 X3 Z7] +
-0.03361644838364539 [X0 X1 X2 Z7] +
0.03422206211800904 [X0 X1 Z5] +
-0.014173745323078241 [X0 X1 Z6] +
0.09518660668118643 [X0 X4] +
-0.03422206211800904 [X0 Z5] +
0.09518660668118643 [X1] +
-0.03361644838364539 [X1 X2 X3 X4 Z7] +
-0.014173745323078241 [X1 X2 X4 Z6] +
0.03361644838364539 [X1 X2 X4 Z7] +
-0.03361644838364539 [X1 X2 Z5 Z6] +
0.014173745323078241 [X1 X2 Z5 Z7] +
-0.03422206211800904 [X1 X4 Z5] +
0.03361644838364539 [X1 Z5 Z6] +
0.09542973454771683 [X2] +
0.014173745323078241 [X2 X3 Z5 Z7] +
-0.03691448415649904 [X2 X3 Z6 Z7] +
0.03361644838364539 [X2 Z5 Z6] +
0.03691448415649904 [X2 Z6 Z7] +
0.09542973454771683 [X3] +
0.03691448415649904 [X3 Z6 Z7] +
0.03422206211800904 [X4 Z5] +
-0.03361644838364539 [Z5 Z6] +
-0.03691448415649904 [Z6 Z7]


In addition, current quantum computer can measure only the $z$ operators. Thus, QWC fragments with $x$ or $y$ operators require extra single-qubit unitaries that rotate them into $z$.  

In [13]:
uz = get_zform_unitary(qwc)
print("Checking whether U * U^+ is identity: {}".format(uz * uz))

allz = remove_complex(uz * qwc * uz)
print("\nThe all-z form of qwc fragment:\n{}".format(allz))

Checking whether U * U^+ is identity: 0.9999999999999998 []

The all-z form of qwc fragment:
-0.2434757373932896 [] +
0.033616448383645386 [Z0 Z1 Z2 Z3 Z7] +
-0.033616448383645386 [Z0 Z1 Z2 Z7] +
0.03422206211800902 [Z0 Z1 Z5] +
-0.014173745323078234 [Z0 Z1 Z6] +
0.09518660668118631 [Z0 Z4] +
-0.03422206211800902 [Z0 Z5] +
0.09518660668118631 [Z1] +
-0.033616448383645386 [Z1 Z2 Z3 Z4 Z7] +
-0.014173745323078234 [Z1 Z2 Z4 Z6] +
0.033616448383645386 [Z1 Z2 Z4 Z7] +
-0.033616448383645386 [Z1 Z2 Z5 Z6] +
0.014173745323078234 [Z1 Z2 Z5 Z7] +
-0.03422206211800902 [Z1 Z4 Z5] +
0.033616448383645386 [Z1 Z5 Z6] +
0.09542973454771668 [Z2] +
0.014173745323078234 [Z2 Z3 Z5 Z7] +
-0.03691448415649901 [Z2 Z3 Z6 Z7] +
0.033616448383645386 [Z2 Z5 Z6] +
0.03691448415649901 [Z2 Z6 Z7] +
0.09542973454771668 [Z3] +
0.03691448415649901 [Z3 Z6 Z7] +
0.03422206211800902 [Z4 Z5] +
-0.033616448383645386 [Z5 Z6] +
-0.03691448415649901 [Z6 Z7]
